# Semantic Segmentation Exercise - I
Data Set: https://www.kaggle.com/bulentsiyah/semantic-drone-dataset
Generate the mask images for Original images. Labeled mask images are available in labeled_images_semantic. 
Hints: 
Need to choose the proper activation function. 
Feel free to add additional layers.
Take 10% test data


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
images_fpath='/kaggle/input/semantic-drone-dataset/dataset/semantic_drone_dataset/original_images/'
masks_fpath='/kaggle/input/semantic-drone-dataset/RGB_color_image_masks/RGB_color_image_masks/'
#masks_fpath= '/kaggle/input/semantic-drone-dataset/dataset/semantic_drone_dataset/label_images_semantic/'

#test_fpath='../input/3d_images/'
#test_masks_fpath='../input/3d_masks/'
print(os.listdir('/kaggle/input/semantic-drone-dataset/dataset/semantic_drone_dataset/original_images/'))
print(os.listdir('/kaggle/input/semantic-drone-dataset/RGB_color_image_masks/RGB_color_image_masks/'))
#print(os.listdir('/kaggle/input/semantic-drone-dataset/dataset/semantic_drone_dataset/label_images_semantic/'))

Start your code here....

In [ ]:
from PIL import Image
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

#for building and training the U-Net model
from tensorflow.keras.layers import Conv2D, Input, Concatenate, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model

print("Loaded all libraries")


def load_images_masks(path_images,path_masks):
    images = os.listdir(path_images)
    masks = os.listdir(path_masks)
    images_list = []
    mask_list = []

    for i in images:
        im = Image.open(path_images + i)
        im1 = np.array(im.resize((128,128)))/255
        images_list.append(im1)
    del im,im1
    for i in masks:
        mask = Image.open(path_masks + i)
        mask1 = np.array(mask.resize((128,128)))/255
        mask_list.append(mask1)
    del mask,mask1
    return images_list,mask_list

In [ ]:
print("No. of images =",len(os.listdir(images_fpath)))
print("No. of image masks =",len(os.listdir(masks_fpath)))

In [ ]:
#1 example let's pick image id '000'
def plot_img_and_mask(id):
    img = cv2.imread(images_fpath + id + '.jpg')
    img_mask = cv2.imread(masks_fpath + id + '.png')
    plt.figure(figsize=(10,10))
    plt.subplot(1,3,1,title='Actual image')
    plt.imshow(img,cmap='gray')
    plt.subplot(1,3,3,title='Image mask')
    plt.imshow(img_mask,cmap='gray')
    
plot_img_and_mask('000')

In [ ]:
plot_img_and_mask('140')

In [ ]:
plot_img_and_mask('160')

def initialize_img_data(folder):
    lst=[] 
    for image in os.listdir(folder):
        #load image in grayscale
        img= cv2.imread(folder+"/"+image)
        #convert to array
        img_array=Image.fromarray(img)
        #resize image
        resize_img = img_array.resize((128 , 128))
        #divide by 255 -> scaling data
        norm_img=np.array(resize_img)/255
        #expand dimensions
        img_array = norm_img.reshape((128,128))#np.expand_dims(norm_img,axis=3)
        lst.append(img_array)
        del img, img_array,resize_img,norm_img
        lst.sort()
    return lst
    


In [ ]:
import re
def sorted_alphanum(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)


In [ ]:
def load_images_masks(path_images,path_masks):
    images = os.listdir(path_images)
    masks = os.listdir(path_masks)
    images_list = []
    masks_list = []
    print(len(images))

    
    images2 = sorted_alphanum(images)
    masks2 = sorted_alphanum(masks)
    
    
    for i in images2:
        im = Image.open(path_images + i)
        im1 = np.array(im.resize((128,128)))/255
        images_list.append(im1)
 
    for i in masks2:
        mask = Image.open(path_masks + i)
        mask1 = np.array(mask.resize((128,128)))/255
        masks_list.append(mask1)
        
    return images_list,masks_list

In [ ]:
X,y=load_images_masks(images_fpath,masks_fpath)

In [ ]:
#quick verification whether all the data is initialized or not
print(len(os.listdir(images_fpath)),len(X),len(y))
print(X[0].shape,y[0].shape)

In [ ]:
#check data once they are loaded
plt.figure(figsize=(10,10))
plt.subplot(1,3,1,title='Original image')
plt.imshow(X[100],cmap='viridis')

plt.subplot(1,3,3,title='Image mask')
plt.imshow(y[100],cmap='viridis')

# Start your code here....

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
#import all libraries for UNET model
from tensorflow.keras.layers import Conv2D, Input, Concatenate, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model

In [ ]:
# Build U-Net model - Define layers
inputs = Input(shape=(128, 128, 3))

c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (inputs)
c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (p4)
c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (c5)

u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = Concatenate()([u6, c4])
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = Concatenate()([u7, c3])
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = Concatenate()([u8, c2])
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = Concatenate()([u9, c1]) #removed ,axis=3
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

outputs = Conv2D(3, (1, 1), activation='relu') (c9)

In [ ]:
model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.1)

In [ ]:
#early_stop = EarlyStopping(patience=5)
#check_point = ModelCheckpoint('model.hdf5',save_best_only=True)
#model.fit(x_train, y_train, epochs=20, callbacks=[early_stop,check_point])
model.fit(x_train, y_train, epochs=100)

In [ ]:
from keras.utils import plot_model
plot_model(model,to_file='model.png')

In [ ]:
x_train.shape

In [ ]:
pred_test = model.predict(x_test, verbose=1)
#print(y_train[0])

In [ ]:
import matplotlib.pyplot as plt

# End your code here...

In [ ]:
plt.figure(1 , figsize = (15, 9))
n = 0 
for i in range(4):
    n += 1 
    #r = np.random.randint(0, x_test.shape[0], 1)
    r=[i]
    plt.subplot(2, 2, n)
    plt.subplots_adjust(hspace = 0.5, wspace = 0.5)
    plt.imshow(x_test[r[0]])
    plt.imshow(pred_test[r[0]], alpha=0.5)
    

plt.suptitle("Annotated images...")
plt.show()

In [ ]:
plt.figure(1 , figsize = (15, 9))
n = 0 
for i in range(6):
    n += 1 
    #r = np.random.randint(0, x_test.shape[0], 1)
    r=[i]
    plt.subplot(3, 2, n)
    plt.subplots_adjust(hspace = 0.5, wspace = 0.5)
    plt.imshow(x_test[r[0]])
    plt.imshow(y_test[r[0]], alpha=0.5)